<a href="https://colab.research.google.com/github/leorex819-creator/fake-news-detection-multilingual_videos/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>